Meta Learning

NumerAi

Tanmay Singh
2021569
CSAI
Class of '25

Importing the Dependencies

In [ ]:
!pip install scipy
!pip install numpy
!pip install pandas
!pip install xgboost
!pip install seaborn
!pip install pyarrow
!pip install numerapi
!pip install imblearn
!pip install catboost
!pip install lightgbm
!pip install matplotlib
!pip install cloudpickle
!pip install mplcyberpunk
!pip install scikit-learn
!pip install torchsummary

In [1]:
import os
import gc
import time
import json
import pickle
import numpy as np
import pandas as pd
import seaborn as sb
import xgboost as xgb
import lightgbm as lgb
import cloudpickle as cp
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [4]:
from torchsummary import summary
from torch.utils.data import DataLoader, Dataset

In [5]:
from tqdm import tqdm
from scipy import stats
from numerapi import NumerAPI
from scipy.stats import pearsonr
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [6]:
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.decomposition import *
from sklearn.preprocessing import *
from sklearn.neural_network import *
from sklearn.model_selection import *
from sklearn.cluster._kmeans import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

Loading the Dataset

In [ ]:
NumerAi = NumerAPI()

In [ ]:
all_datasets = NumerAi.list_datasets()
dataset_versions = list(set(d.split('/')[0] for d in all_datasets))
print("Available versions:\n", dataset_versions)

In [ ]:
DATA_VERSION = "v5.0"

current_version_files = [f for f in all_datasets if f.startswith(DATA_VERSION)]
print("availbable", DATA_VERSION, "files:\n", current_version_files)

Creating a Feature Set

In [ ]:
NumerAi.download_dataset(f"{DATA_VERSION}/features.json")

feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
for metadata in feature_metadata:
  print(metadata, len(feature_metadata[metadata]))

In [ ]:
feature_sets = feature_metadata["feature_sets"]
for feature_set in ["small", "medium", "all"]:
  print(feature_set, len(feature_sets[feature_set]))

In [ ]:
feature_sets = feature_metadata["feature_sets"]
feature_sets.keys()

In [ ]:
for feature_set in feature_sets:
  print(f'Feature Set: {feature_set:<25}', f'Size: {len(feature_sets[feature_set])}')

Loading the Saved Experts & the Meta-Model

In [24]:
with open('numerai_expert1.pkl', 'rb') as f:
    expert1 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [25]:
with open('numerai_expert2.pkl', 'rb') as f:
    expert2 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [26]:
with open('numerai_expert3.pkl', 'rb') as f:
    expert3 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [27]:
with open('numerai_expert4.pkl', 'rb') as f:
    expert4 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [28]:
with open('numerai_expert5.pkl', 'rb') as f:
    expert5 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [29]:
with open('numerai_expert6.pkl', 'rb') as f:
    expert6 = pickle.load(f)
print("Model loaded successfully!")

Model loaded successfully!


In [31]:
with open('numerai_meta_model.pkl', 'rb') as f:
    meta_model = pickle.load(f)
print("Meta Model loaded successfully!")

Meta Model loaded successfully!


Loading the Validation Set, with a 'medium' feature set

In [ ]:
feature_set = feature_sets["medium"]

NumerAi.download_dataset(f"{DATA_VERSION}/validation.parquet")

val = pd.read_parquet(
    f"{DATA_VERSION}/validation.parquet",
    columns=["era", "target"] + feature_set
)


Preprocessing the Validation Set (in the same manner as the Training Set)

In [ ]:
val.rename(columns=lambda x: f'feature {feature_set.index(x)}' if x in feature_set else x, inplace=True)
feature_set = val.columns.drop(["era", "target"])

In [ ]:
val['era'] = val['era'].astype('int32')

In [7]:
val = pd.read_parquet(f"../data/prepared/validation.parquet")

In [9]:
feature_set = val.columns.drop(["era", "target"])

In [8]:
val

,era,target,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n000101811a8a843,575,0.75,2,1,2,4,0,4,0,0,...,3,0,0,3,4,2,0,0,1,0
n001e1318d5072ac,575,0.00,0,2,2,3,2,3,0,0,...,3,2,3,0,1,1,0,3,4,3
n002a9c5ab785cbb,575,0.50,0,2,2,3,3,3,1,1,...,2,1,2,0,2,3,3,4,2,2
n002ccf6d0e8c5ad,575,0.00,0,1,1,4,3,4,0,4,...,1,1,4,2,0,0,1,3,1,3
n0041544c345c91d,575,0.50,4,2,0,0,3,3,3,4,...,2,3,4,3,2,4,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffad4c8e7a7f4ba,1129,0.25,0,1,1,1,3,2,1,4,...,2,0,3,4,2,4,2,2,1,2
nffc1734f40562ce,1129,0.50,4,1,4,1,1,1,4,2,...,0,3,0,4,1,0,4,0,0,4
nffc7ab97c534638,1129,0.50,3,4,1,4,0,2,3,3,...,4,3,1,3,1,1,4,1,3,0


In [ ]:
unique_era = val['era'].unique()

In [ ]:
val[val['era'] == unique_era[0]]

In [11]:
test_set = val
test_set

,era,target,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n000101811a8a843,575,0.75,2,1,2,4,0,4,0,0,...,3,0,0,3,4,2,0,0,1,0
n001e1318d5072ac,575,0.00,0,2,2,3,2,3,0,0,...,3,2,3,0,1,1,0,3,4,3
n002a9c5ab785cbb,575,0.50,0,2,2,3,3,3,1,1,...,2,1,2,0,2,3,3,4,2,2
n002ccf6d0e8c5ad,575,0.00,0,1,1,4,3,4,0,4,...,1,1,4,2,0,0,1,3,1,3
n0041544c345c91d,575,0.50,4,2,0,0,3,3,3,4,...,2,3,4,3,2,4,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffad4c8e7a7f4ba,1129,0.25,0,1,1,1,3,2,1,4,...,2,0,3,4,2,4,2,2,1,2
nffc1734f40562ce,1129,0.50,4,1,4,1,1,1,4,2,...,0,3,0,4,1,0,4,0,0,4
nffc7ab97c534638,1129,0.50,3,4,1,4,0,2,3,3,...,4,3,1,3,1,1,4,1,3,0


In [12]:
test_set.isna().any().any()
test_set['target'].value_counts()

target
0.50    1713473
0.25     687577
0.75     687000
1.00     170012
0.00     169645
Name: count, dtype: int64

Encoding the Numeric Values in the Target into corresponding labels (class 0 to class 4)

In [13]:
label_encoder = LabelEncoder()
label_encoder.fit(test_set['target'])
test_set['target'] = label_encoder.transform(test_set['target'])

In [15]:
labels_dict = {i: label_encoder.transform([i])[0] for i in label_encoder.classes_}
labels_dict

{0.0: 0, 0.25: 1, 0.5: 2, 0.75: 3, 1.0: 4}

In [16]:
test_df_x = test_set.drop(['target'], axis=1, inplace=False)
test_df_y = test_set['target']

In [17]:
test_df_x

,era,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n000101811a8a843,575,2,1,2,4,0,4,0,0,2,...,3,0,0,3,4,2,0,0,1,0
n001e1318d5072ac,575,0,2,2,3,2,3,0,0,4,...,3,2,3,0,1,1,0,3,4,3
n002a9c5ab785cbb,575,0,2,2,3,3,3,1,1,2,...,2,1,2,0,2,3,3,4,2,2
n002ccf6d0e8c5ad,575,0,1,1,4,3,4,0,4,0,...,1,1,4,2,0,0,1,3,1,3
n0041544c345c91d,575,4,2,0,0,3,3,3,4,2,...,2,3,4,3,2,4,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffad4c8e7a7f4ba,1129,0,1,1,1,3,2,1,4,0,...,2,0,3,4,2,4,2,2,1,2
nffc1734f40562ce,1129,4,1,4,1,1,1,4,2,4,...,0,3,0,4,1,0,4,0,0,4
nffc7ab97c534638,1129,3,4,1,4,0,2,3,3,3,...,4,3,1,3,1,1,4,1,3,0


In [18]:
test_df_y

id
n000101811a8a843    3
n001e1318d5072ac    0
n002a9c5ab785cbb    2
n002ccf6d0e8c5ad    0
n0041544c345c91d    2
                   ..
nffad4c8e7a7f4ba    1
nffc1734f40562ce    2
nffc7ab97c534638    2
nffd95eda4b80ac3    2
nffe2bb826bd8ffc    1
Name: target, Length: 3427707, dtype: int64

**************************************************************************************************************************************
"DUE TO RESOURCE CONSTRAINTS, THIS FILE WAS RUN WITH AT MAX 10 LAKH ENTRIES FOR 4 TIMES, ACCOUNTING TO THE SIZE OF THE VALIDATION SET"
**************************************************************************************************************************************

In [19]:
test_df_x_resampled = test_df_x          #Run-1
# test_df_x_resampled = test_df_x[1000000:2000000] #Run-2
# test_df_x_resampled = test_df_x[2000000:3000000] #Run-3
# test_df_x_resampled = test_df_x[3000000:]        #Run-4

test_df_x_resampled

,era,feature 0,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,...,feature 695,feature 696,feature 697,feature 698,feature 699,feature 700,feature 701,feature 702,feature 703,feature 704
id,,,,,,,,,,,,,,,,,,,,,
n000101811a8a843,575,2,1,2,4,0,4,0,0,2,...,3,0,0,3,4,2,0,0,1,0
n001e1318d5072ac,575,0,2,2,3,2,3,0,0,4,...,3,2,3,0,1,1,0,3,4,3
n002a9c5ab785cbb,575,0,2,2,3,3,3,1,1,2,...,2,1,2,0,2,3,3,4,2,2
n002ccf6d0e8c5ad,575,0,1,1,4,3,4,0,4,0,...,1,1,4,2,0,0,1,3,1,3
n0041544c345c91d,575,4,2,0,0,3,3,3,4,2,...,2,3,4,3,2,4,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffad4c8e7a7f4ba,1129,0,1,1,1,3,2,1,4,0,...,2,0,3,4,2,4,2,2,1,2
nffc1734f40562ce,1129,4,1,4,1,1,1,4,2,4,...,0,3,0,4,1,0,4,0,0,4
nffc7ab97c534638,1129,3,4,1,4,0,2,3,3,3,...,4,3,1,3,1,1,4,1,3,0


In [20]:
test_df_y_resampled = test_df_y          #Run-1
# test_df_y_resampled = test_df_y[1000000:2000000] #Run-2
# test_df_y_resampled = test_df_y[2000000:3000000] #Run-3
# test_df_y_resampled = test_df_y[3000000:]        #Run-4

test_df_y_resampled

id
n000101811a8a843    3
n001e1318d5072ac    0
n002a9c5ab785cbb    2
n002ccf6d0e8c5ad    0
n0041544c345c91d    2
                   ..
nffad4c8e7a7f4ba    1
nffc1734f40562ce    2
nffc7ab97c534638    2
nffd95eda4b80ac3    2
nffe2bb826bd8ffc    1
Name: target, Length: 3427707, dtype: int64

Function to compute Label Frequencies

In [21]:
def neutralize_predictions(predictions: np.ndarray, x: pd.DataFrame, features_to_neutralize: list, proportion: float = 1.0) -> np.ndarray:
	predictions_df = pd.DataFrame(predictions, columns=['prediction'], index=x.index)
	neutralized_preds = predictions_df['prediction'].copy()
	
	for feature in features_to_neutralize:
		neutralizer = x[feature]
		adjustment = proportion * (neutralizer.dot(predictions_df['prediction']) / neutralizer.dot(neutralizer)) * neutralizer
		neutralized_preds -= adjustment
	# neutralizer = x
	# print(neutralizer.shape, neutralized_preds.shape)
	# adjustment = proportion * (neutralizer.dot(predictions_df['prediction']) / neutralizer.dot(neutralizer)) * neutralizer
	# neutralized_preds -= adjustment
   
	return neutralized_preds.values

In [22]:
def label_frequency(predictions):
    unique, counts = np.unique(predictions, return_counts=True)
    label_frequencies = dict(zip(unique, counts))
    print("Label frequencies:", label_frequencies)

Function to compute NumerAi Correlation

In [23]:
def numerai_corr(preds, target):
  ranked_preds = (preds.rank(method="average").values - 0.5) / preds.count()
  gauss_ranked_preds = stats.norm.ppf(ranked_preds)

  centered_target = target - target.mean()

  preds_p15 = np.sign(gauss_ranked_preds) * np.abs(gauss_ranked_preds) ** 1.5
  target_p15 = np.sign(centered_target) * np.abs(centered_target) ** 1.5

  return np.corrcoef(preds_p15, target_p15)[0, 1]

Generating Predictions from Experts

EXPERT-1 (XGBOOST CLASSIFIER)

In [37]:
expert1_test_pred = expert1.predict(test_df_x_resampled)

In [38]:
expert1_test_pred

array([1, 3, 2, ..., 2, 3, 3])

In [42]:
np.save('expert1_test_pred.npy', expert1_test_pred)

In [ ]:
expert1_test_pred = neutralize_predictions(predictions=expert1_test_pred, x=test_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert1:", expert1_test_pred)

EXPERT-2 (RANDOM FOREST CLASSIFIER)

In [35]:
expert2_test_pred = expert2.predict(test_df_x_resampled)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    3.7s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   19.4s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:   46.4s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:  1.4min
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:  1.8min finished


In [36]:
expert2_test_pred

array([2., 2., 2., ..., 2., 2., 0.], dtype=float32)

In [43]:
np.save('expert2_test_pred.npy', expert2_test_pred)

In [ ]:
expert2_test_pred = neutralize_predictions(predictions=expert2_test_pred, x=test_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert2:", expert2_test_pred)

EXPERT-3 (ADABOOST CLASSIFIER with DECISION TREE CLASSIFIER as BASE ESTIMATOR)

In [39]:
expert3_test_pred = expert3.predict(test_df_x_resampled)

In [40]:
expert3_test_pred

array([2., 3., 2., ..., 2., 3., 3.], dtype=float32)

In [41]:
np.save("expert3_test_pred.npy", expert3_test_pred)

In [ ]:
expert3_test_pred = neutralize_predictions(predictions=expert3_test_pred, x=test_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert3:", expert3_test_pred)

EXPERT-4 (LOGISTIC REGRESSION)

In [44]:
expert4_test_pred = expert4.predict(test_df_x_resampled)

In [45]:
expert4_test_pred

array([2., 3., 2., ..., 2., 3., 2.], dtype=float32)

In [46]:
np.save("expert4_test_pred.npy", expert4_test_pred)

In [ ]:
expert4_test_pred = neutralize_predictions(predictions=expert4_test_pred, x=test_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert4:", expert4_test_pred)

EXPERT-5 (CATBOOST CLASSIFIER)

In [47]:
expert5_test_pred = expert5.predict(test_df_x_resampled)

In [48]:
if expert5_test_pred.ndim > 1:
    expert5_test_pred = expert5_test_pred.ravel()

In [49]:
expert5_test_pred

array([2., 3., 2., ..., 2., 3., 3.])

In [50]:
np.save("expert5_test_pred.npy", expert5_test_pred)

In [ ]:
expert5_test_pred = neutralize_predictions(predictions=expert5_test_pred, x=test_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert5:", expert5_test_pred)

EXPERT-6 (HISTOGRAM-BASED GRADIENT BOOST CLASSIFIER)

In [51]:
expert6_test_pred = expert6.predict(test_df_x_resampled)

In [52]:
expert6_test_pred

array([2., 3., 2., ..., 3., 3., 2.], dtype=float32)

In [53]:
np.save("expert6_test_pred.npy", expert6_test_pred)

In [ ]:
expert6_test_pred = neutralize_predictions(predictions=expert6_test_pred, x=test_df_x_resampled, features_to_neutralize=feature_set, proportion=0.0001)
print("Neutralized predictions, Expert6:", expert6_test_pred)

Generating Predictions from the Meta-Model (LIGHTGBM)

In [54]:
meta_test_x = np.column_stack((expert1_test_pred, expert2_test_pred, expert3_test_pred, expert4_test_pred, expert5_test_pred, expert6_test_pred))

In [55]:
meta_test_x

array([[1., 2., 2., 2., 2., 2.],
       [3., 2., 3., 3., 3., 3.],
       [2., 2., 2., 2., 2., 2.],
       ...,
       [2., 2., 2., 2., 2., 3.],
       [3., 2., 3., 3., 3., 3.],
       [3., 0., 3., 2., 3., 2.]])

In [56]:
meta_test_y_pred = meta_model.predict(meta_test_x)

In [57]:
meta_test_y_pred

array([1.69097534, 2.37978199, 2.41139138, ..., 2.31735773, 2.37978199,
       2.33748241])

In [58]:
np.save("meta_test_y_pred.npy", meta_test_y_pred)

In [59]:
bins = [0.5, 1.5, 2.5, 3.5]

rounded_predictions = np.digitize(meta_test_y_pred, bins)

In [60]:
rounded_predictions

array([2, 2, 2, ..., 2, 2, 2])

In [61]:
label_frequency(rounded_predictions)

Label frequencies: {1: 506285, 2: 2090820, 3: 830602}


In [62]:
test_df_y_resampled

id
n000101811a8a843    3
n001e1318d5072ac    0
n002a9c5ab785cbb    2
n002ccf6d0e8c5ad    0
n0041544c345c91d    2
                   ..
nffad4c8e7a7f4ba    1
nffc1734f40562ce    2
nffc7ab97c534638    2
nffd95eda4b80ac3    2
nffe2bb826bd8ffc    1
Name: target, Length: 3427707, dtype: int64

Computing Relevant Evaluation Metrics

In [63]:
acc = accuracy_score(rounded_predictions, test_df_y_resampled)
print("Accuracy on Validation Set: ", acc)

Accuracy on Validation Set:  0.43386584675994766


Pearson's Correlation

In [64]:
pearson_corr, _ = stats.pearsonr(rounded_predictions, test_df_y_resampled)
print("Pearson Correlation:", pearson_corr)

Pearson Correlation: 0.016454382504250707


Saving the Predictions in a Pickle File

RUN-1

In [ ]:
with open('numerai_prediction1.pkl', 'wb') as f:
    pickle.dump(rounded_predictions, f)

print("Predictions saved successfully to numerai_prediction1.pkl!")

RUN-2

In [ ]:
# with open('numerai_prediction2.pkl', 'wb') as f:
#     pickle.dump(rounded_predictions, f)

# print("Predictions saved successfully to numerai_prediction2.pkl!")

RUN-3

In [ ]:
# with open('numerai_prediction3.pkl', 'wb') as f:
#     pickle.dump(rounded_predictions, f)

# print("Predictions saved successfully to numerai_prediction3.pkl!")

RUN-4

In [ ]:
# with open('numerai_prediction4.pkl', 'wb') as f:
#     pickle.dump(rounded_predictions, f)

# print("Predictions saved successfully to numerai_prediction4.pkl!")

In [65]:
with open('numerai_prediction_full.pkl', 'wb') as f:
    pickle.dump(rounded_predictions, f)

print("Predictions saved successfully to numerai_prediction_full.pkl!")

Predictions saved successfully to numerai_prediction_full.pkl!


Computing the NumerAi's Correlation Metric

In [66]:
rounded_predictions = pd.Series(rounded_predictions)

In [67]:
actual_corr = numerai_corr(rounded_predictions, test_df_y_resampled)
actual_corr

0.016540745941191076